# 摘要

在很多预测模型中，feature engineering是非常关键的一个步骤。然而，特征工程往往非常不容易，因为经常需要hand-craft来设计特征，无法穷尽所有尽可能的特征情况。DNN模型具有天然特征组合的能力，但是它对特征的组合是隐式的，在学习所有类型的cross features上并不高效。

因此本文提出Deep&Cross Network(DCN)，不仅保留了DNN模型的优势，并且提出一种新的有效学习特定有限维度特征交叉的能力（certain bounded-degree）。DCN在几乎不增加复杂度的情况下，每一层都显示地应用了特征交叉。

# 简介

CTR预估对于广告行业是非常重要的，要预估好CTR的关键是发现相关的特征，同时探索潜在的、不常见的交叉特征也非常重要。然而在CTR预估中数据往往是高维离散特征，这无异于给特征探索带来了巨大困难。这也使得大部分情况下都使用大规模分布式LR来进行建模，但LR本身的表达能力有限，且需要人工构造特征，无法自动地去探索特征组合。

因此，本文提出一种新的神经网络结构 - cross network，这种网络可以显式地、自动地进行特征交叉。cross network具有多层结构，特征交叉的最高阶数取决于网络层数；在cross network中，每一层都会在当前层的基础上对特征交叉阶数进行提升。

除此之外，模型加入DNN部分，与cross network进行jointly training。DNN具备复杂特征交叉的能力，但相比于cross network，它需要几乎多出一个数量级的参数；其次，DNN无法显式地进行特征交叉，因此对于学习特征类型的特征交叉能力较差。

## 相关工作

随着数据量和数据维度的增大，许多模型都朝着embedding和NN方向进行发展。
- FM将sparse feature投影到低维dense vectors，再通过向量内积学习交叉特征
- FFM在FM的基础上引入field概念，增加模型对于不同field的表达能力
- FM和FFM只能进行低阶的交叉特征学习，如果想进行高阶特征学习，计算消耗会增加
- DNN具备学习high-degree feature interactions的能力
- Deep Crossing 对残差网络进行了扩展，通过堆叠所有类型的输入实现了自动特征学习

DNN对于特征的学习是隐式的、高度非线性的，这就启发我们，可以设计一个比DNN更有效地、显式地具备学习有限阶（bounded-degree）特征交叉的网络模型。wide&deep可以提供一些参考，它将特征交叉作为wide线性部分的输入，并与DNN模型jointly training，但w&d的成功取决于合适的交叉特征，这一问题（选取合适的cross features）尚未解决。

## 主要贡献

本文提出Deep & Cross Network(DCN)，可以自动地、有效地对sparse和dense inputs进行有限维阶交叉特征学习，并且不需要人工设计特征工程，同时具备较低的计算消耗，主要贡献有：
- 提出新的cross network显式地自动学习有限阶特征交叉
- cross network简单且高效，最高阶特征交叉取决于网络层数，考虑了从低阶到高阶的所有特征交叉模式
- cross network内存高效且易于实现
- 相比于DNN，DCN能够在少一个数量级参数的情况下，达到更低的logloss

# DCN模型

DCN网络包含：
- Embedding and Stacking Layer
- cross network
- deep network
- combination layer

![](pictures/dcn.png)

## Embedding与Stacking层

特征部分包括sparse特征和dense特征：
- sparse特征：Embedding，$\mathbb{x}_{embed,i}=W_{embed,i}x_i$
- dense特征：log transformation

两部分特征处理完后，将所有特征concat起来获得输入$\mathbb{x}_0=[\mathbb{x}_{embed,1}^T, \mathbb{x}_{embed,2}^T,\cdots, \mathbb{x}_{dense}^T]$

## Cross Network

Cross Network的核心是以一种高效的方式应用显式地特征交叉。

对于cross network中的每一层，有：

$$\mathbb{x}_{l+1}=\mathbb{x}_0\mathbb{x}_l^T w_l+b_l+\mathbb{x}_l=f(\mathbb{x}_l,w_l,b_l)+\mathbb{x}_l$$

其中，$\mathbb{x}_l,\mathbb{x}_{l+1}\in \mathbb{R}^d$，我们可以发现，mapping function $f: \mathbb{R}^d \rightarrow \mathbb{R}^d$正好拟合了$l+1$层与$l$层的残差：$\mathbb{x}_{l+1}-\mathbb{x}_l$

High-degree Interaction Across Features：

![](pictures/dcn-cross-layer.png)

对于cross network的第$l$层来说，其最高阶是$l+1$；cross network包含了输入向量中的所有从1到$l+1$的交叉项：$x_1^{\alpha_1}x_2^{\alpha_2}\cdots x_d^{\alpha_d}$。

假设$L_c$代表cross layers的个数，$d$代表input dimensions，那么cross network中的所有参数个数为：$d\times L_c\times2$。因此cross network的时间与空间复杂度都是与input dim呈线性关系。

考虑到cross network中的少量参数限制了model capacity，为了捕捉高阶非线性交叉，同时引入deep network。

## Deep Network

Deep Network主要为了捕捉高阶非线性交叉信息，由fully-connected layer组成。

$$h_{l+1}=f(W_l h_l)+b_l$$

其中激活函数为ReLU。

假设deep部分每层拥有同样个数的隐藏层结点$m$，拥有$L_d$层，那么deep network中的参数个数为：$d\times m+m+(m^2+m)\times (L_d-1)$

## Combination Layer

Combination Layer就是将cross network和deep network部分的输出进行concat后，输入logits layer经过sigmoid激活输出。

$$p=\sigma([\mathbb{x}_{L_1}^T, \mathbb{h}_{L_2}^T]W_{logits})$$

其中$\mathbb{x}_{L_1}\in \mathbb{R}^d$，$\mathbb{h}_{L_2}\in \mathbb{R}^m$。

损失函数采用logloss+frobenius范数，最后对cross和deep部分进行jointly training。